In [1]:
# If running in a notebook for the first time:
%pip install openai python-dotenv ipywidgets 
# --quiet


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# --- IMPORTS ---
import os
import json
import time
import openai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")
openai.api_key = api_key

# --- GAME STATE ---
model_name = "gpt-3.5-turbo"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION --- (Updated)
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )
    
    for attempt in range(5):
        try:
            # Use openai.Completion.create for OpenAI API >= 1.0.0
            response = openai.Completion.create(
                model=model_name,
                prompt=prompt,
                max_tokens=150,
                temperature=0.9,
                top_p=0.95
            )
            return response.choices[0].text.strip()  # Updated to use .text for the response
        except openai.error.RateLimitError:
            time.sleep(2 ** attempt)  # Exponential backoff for rate limiting
        except Exception as e:
            return f"❌ Error generating story: {e}"
    return "⚠️ The story could not be continued due to repeated errors."

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')

output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# Display all widgets
start_new_game()
display(input_box, submit_button, output_area)


Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

AttributeError: module 'openai' has no attribute 'error'

Probeersel 2

In [6]:
%pip install --upgrade openai

   ---------------------------------------- 0.0/662.0 kB ? eta -:--:--
   --------------------------------------- 662.0/662.0 kB 12.9 MB/s eta 0:00:00
  Attempting uninstall: openai
    Found existing installation: openai 1.74.0
    Uninstalling openai-1.74.0:
      Successfully uninstalled openai-1.74.0
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [7]:
# --- IMPORTS ---
import os
import json
import time
import openai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")
openai.api_key = api_key

# --- GAME STATE ---
model_name = "gpt-3.5-turbo"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION --- (Updated for v1.0.0)
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )
    
    for attempt in range(5):
        try:
            # Use the new `openai.Chat.create` for the updated API
            response = openai.Chat.create(
                model=model_name,
                messages=[
                    {"role": "system", "content": "You are a helpful assistant."},
                    {"role": "user", "content": prompt}
                ],
                max_tokens=150,
                temperature=0.9,
                top_p=0.95
            )
            return response["choices"][0]["message"]["content"].strip()  # Updated for new API format
        except openai.error.RateLimitError:
            time.sleep(2 ** attempt)  # Exponential backoff for rate limiting
        except Exception as e:
            return f"❌ Error generating story: {e}"
    return "⚠️ The story could not be continued due to repeated errors."

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')

output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# Display all widgets
start_new_game()
display(input_box, submit_button, output_area)


Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

AttributeError: module 'openai' has no attribute 'error'

Probeersel 3

In [17]:
# --- IMPORTS ---
import os
import time
from openai import OpenAI, OpenAIError, RateLimitError
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("❌ OPENAI_API_KEY not found in .env file!")

client = OpenAI(api_key=api_key)

# --- GAME STATE ---
model_name = "gpt-3.5-turbo"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION (OpenAI SDK v1) ---
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )

    for attempt in range(5):
        try:
            response = client.chat.completions.create(
                model=model_name,
                messages=[{"role": "user", "content": prompt}],
                max_tokens=150,
                temperature=0.9,
                top_p=0.95
            )
            return response.choices[0].message.content.strip()
        except RateLimitError:
            time.sleep(2 ** attempt)
        except OpenAIError as e:
            return f"❌ Error generating story: {e}"

    return "⚠️ The story could not be continued due to repeated errors."

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')
output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# --- START & DISPLAY ---
start_new_game()
display(input_box, submit_button, output_area)



Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

test

In [18]:
from openai import OpenAI

client = OpenAI()

response = client.chat.completions.create(
    model="gpt-3.5-turbo",
    messages=[{"role": "user", "content": "Say hello"}],
    max_tokens=10
)

print(response.choices[0].message.content)

RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [19]:
%pip install -q -U google-genai

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [20]:
from google import genai
from dotenv import load_dotenv
import os

# Load .env file
load_dotenv()

# Get the API key from environment variable
api_key = os.getenv("GOOGLE_API_KEY")

if not api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file!")

# Initialize Gemini client
client = genai.Client(api_key=api_key)

# Make a request to Gemini
response = client.models.generate_content(
    model="gemini-2.0-flash",
    contents="Explain how AI works in a few words"
)

print(response.text)


AI learns patterns from data to make predictions or decisions.



Probeersel met google AI of gemini

In [22]:
%pip install google-generativeai

  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ------------------------------- -------- 1.0/1.3 MB 5.6 MB/s eta 0:00:01
   ---------------------------------------- 1.3/1.3 MB 5.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/13.3 MB ? eta -:--:--
   --- ------------------------------------ 1.0/13.3 MB 5.6 MB/s eta 0:00:03
   ------- -------------------------------- 2.4/13.3 MB 5.6 MB/s eta 0:00:02
   ---------- ----------------------------- 3.4/13.3 MB 5.4 MB/s eta 0:00:02
   ------------- -------------------------- 4.5/13.3 MB 5.5 MB/s eta 0:00:02
   ---------------- ----------------------- 5.5/13.3 MB 5.4 MB/s eta 0:00:02
   -------------------- ------------------- 6.8/13.3 MB 5.5 MB/s eta 0:00:02
   ----------------------- ---------------- 7.9/13.3 MB 5.5 MB/s eta 0:00:01
   --------------------------- ------------ 9.2/13.3 MB 5.5 MB/s eta 0:00:01
   ------------------------


[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [23]:
# --- IMPORTS ---
import os
import time
import google.generativeai as genai
from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown

# --- LOAD API KEY ---
load_dotenv()
api_key = os.getenv("GOOGLE_API_KEY")
if not api_key:
    raise ValueError("❌ GOOGLE_API_KEY not found in .env file!")

genai.configure(api_key=api_key)

# --- GAME STATE ---
model_name = "gemini-1.5-pro"
player_name = "Ihno"
context = ""
game_memory = []
player_stats = {}
inventory = []
difficulty = 1

# --- DIFFICULTY ADJUSTMENT ---
def adjust_difficulty(player_input):
    global difficulty
    if any(word in player_input.lower() for word in ["attack", "fight", "battle"]):
        difficulty = min(difficulty + 1, 5)
    elif any(word in player_input.lower() for word in ["run", "talk", "hide"]):
        difficulty = max(difficulty - 1, 1)
    return difficulty

# --- STORY GENERATION (Gemini) ---
def generate_story(context, player_input, difficulty):
    prompt = (
        "You are a fantasy storytelling AI. "
        "Continue the adventure in a vivid, immersive style. "
        f"Difficulty level: {difficulty}.\n\n"
        f"{context}\n"
        f"{player_name}: {player_input}\n"
        "Narrator:"
    )

    try:
        model = genai.GenerativeModel(model_name)
        response = model.generate_content(prompt)
        return response.text.strip()
    except Exception as e:
        return f"❌ Error generating story: {e}"

# --- PRINT GAME STATE ---
def print_game_state():
    display(Markdown(f"### 📖 **Story so far**\n{context}"))
    display(Markdown(f"**🧍 {player_name}'s Inventory:** {inventory}"))
    display(Markdown(f"**❤️ Stats:** {player_stats} | 🎯 Difficulty:** {['Easy', 'Medium', 'Hard', 'Very Hard', 'Nightmare'][difficulty - 1]}"))

# --- PLAY TURN ---
def play_turn(player_input):
    global context
    if not player_input.strip():
        return
    game_memory.append(f"{player_name}: {player_input}")
    adjust_difficulty(player_input)

    recent_context = "\n".join(game_memory[-6:])
    new_story = generate_story(recent_context, player_input, difficulty)
    context_update = f"\n{player_name}: {player_input}\n{new_story}"
    context += context_update
    game_memory.append(new_story)

    output_area.clear_output(wait=True)
    with output_area:
        print_game_state()

# --- START GAME ---
def start_new_game():
    global context, player_stats, inventory, difficulty, game_memory
    context = f"{player_name} awakens in a dark forest. A mysterious figure approaches."
    game_memory = [context]
    player_stats = {"health": 100, "strength": 10, "gold": 5}
    inventory = ["torch"]
    difficulty = 1
    output_area.clear_output()
    with output_area:
        display(Markdown("🆕 **New game started.**"))
        print_game_state()

# --- UI SETUP ---
input_box = widgets.Text(
    placeholder='What does Ihno do next?',
    description='▶️ Action:',
    layout=widgets.Layout(width='70%')
)
submit_button = widgets.Button(description="Submit", button_style='success')
output_area = widgets.Output()

def on_button_click(b):
    play_turn(input_box.value)
    input_box.value = ''

submit_button.on_click(on_button_click)

# --- START & DISPLAY ---
start_new_game()
display(input_box, submit_button, output_area)


Text(value='', description='▶️ Action:', layout=Layout(width='70%'), placeholder='What does Ihno do next?')

Button(button_style='success', description='Submit', style=ButtonStyle())

Output()

Token conuter for prompts

In [25]:
%pip install tiktoken

   ---------------------------------------- 0.0/893.9 kB ? eta -:--:--
   ---------------------------------------- 893.9/893.9 kB 5.1 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [26]:
import tiktoken

# Define the prompt text
prompt_text = """
Ihno awakens in a dark forest. A mysterious figure approaches. Ihno: I talk with the mysterious figure. Narrator: The forest floor, a tapestry of damp earth and decaying leaves, pressed against Ihno's cheek. He blinked, disoriented, the canopy of interwoven branches obscuring the sky. Emerald twilight filtered through the leaves, painting the world in an ethereal green glow. He sat up, pushing himself to his feet, his head throbbing with a dull ache. He remembered nothing.

A rustling in the undergrowth made him jump. A figure emerged from the shadows, its form cloaked in a dark, hooded robe that trailed along the forest floor. The hood obscured its face, leaving only a sliver of pale skin visible beneath. It carried a long, gnarled staff, its top carved into the shape of a crescent moon.

The figure stopped a few paces from Ihno, silent and still. The air grew heavy with an unspoken magic.

Ihno swallowed, his throat dry. "Who are you?" he asked, his voice a shaky whisper that seemed swallowed by the vastness of the woods.

The figure raised a hand, a slow, deliberate movement. It pointed the staff towards a path barely discernible through the dense foliage. A faint, silvery light seemed to emanate from the staff's tip, illuminating the path like a moonbeam.

A low, melodious voice, like the rustling of leaves in a gentle breeze, echoed around Ihno. "The path awaits," it said. "It will lead you to the answers you seek."
"""

# Initialize the tokenizer for GPT models
encoder = tiktoken.get_encoding("gpt2")

# Tokenize the text
tokens = encoder.encode(prompt_text)

# Count the number of tokens
num_tokens = len(tokens)
print(f"Number of tokens in the prompt: {num_tokens}")


Number of tokens in the prompt: 351
